# 📒 **JAX Pruebas**

En este cuaderno se realizarán pruebas prácticas con **JAX**, una librería desarrollada por Google para computación numérica y aprendizaje automático de alto rendimiento. El objetivo es explorar sus principales características, como la diferenciación automática, la compilación just-in-time (JIT), la vectorización y la ejecución acelerada en CPU/GPU/TPU.

A lo largo del cuaderno se implementarán ejemplos que permitan:

* Comprender el funcionamiento básico de JAX.
* Comparar su estilo de programación con frameworks como TensorFlow y PyTorch.
* Evaluar su rendimiento y facilidad de uso en tareas de aprendizaje automático.

<img src="https://repository-images.githubusercontent.com/154739597/90607180-e100-11e9-8642-c65819bec604" width="150"/>

**Autor: [Alejandro Barrionuevo Rosado](https://github.com/Alejandro-BR)**

Máster de FP en Inteligencia Artifical y Big Data - CPIFP Alan Turing *texto en cursiva*
<!-- <img src="./img/alan_turing.png" width="150"/> -->

[![Jax Research](https://img.shields.io/badge/jax--research-GitHub-181717?style=flat&logo=github&logoColor=white)](https://github.com/Alejandro-BR/jax-research)
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Alejandro-BR/jax-research/blob/main/notebooks/jax_tests.ipynb)
![Python](https://img.shields.io/badge/python-3.10-blue?style=flat&logo=python&logoColor=white)



## **Definición**

JAX es una biblioteca de Python desarrollada por Google para aprendizaje automático y computación numérica de alto rendimiento. Su API se basa en **NumPy**, lo que permite trabajar con funciones numéricas de manera familiar y sencilla. Gracias a esto, JAX resulta **flexible, fácil de aprender y eficiente** para realizar cálculos avanzados en CPU, GPU o TPU.


## **Instalación de dependencias**

En esta sección se instalarán todas las dependencias que utilizaremos en este cuaderno.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### **Instalación JAX**

<img src="https://repository-images.githubusercontent.com/154739597/90607180-e100-11e9-8642-c65819bec604" width="150"/>

CPU-only (Linux/macOS/Windows)
```python
!pip install -U jax
```
GPU (NVIDIA, CUDA 13)
```python
!pip install -U "jax[cuda13]"
```
TPU (Google Cloud TPU VM)
```python
!pip install -U "jax[tpu]"
```

[Instalación JAX](https://docs.jax.dev/en/latest/installation.html)

In [1]:
# CPU-only (Linux/macOS/Windows)
!pip install -U jax
# GPU (NVIDIA, CUDA 13)
# !pip install -U "jax[cuda13]"
# TPU (Google Cloud TPU VM)
# !pip install -U "jax[tpu]"

import jax
import jax.numpy as jnp

print("JAX version:", jax.__version__)
print("Backend (CPU/GPU/TPU):", jax.devices())

JAX version: 0.9.0
Backend (CPU/GPU/TPU): [CpuDevice(id=0)]


### **Instalación de TensorFlow**

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/ab/TensorFlow_logo.svg/1280px-TensorFlow_logo.svg.png" width="150"/>




In [5]:
import tensorflow as tf
from tensorflow import keras

print("Versión de TensorFlow:", tf.__version__)
print("GPU disponible:", "Sí" if tf.config.list_physical_devices('GPU') else "No")
print("Versión de Keras:", keras.__version__)

Versión de TensorFlow: 2.19.0
GPU disponible: No
Versión de Keras: 3.10.0


### **Instalación de PyTorch**

<img src="https://opencv.org/wp-content/uploads/2024/01/tensorflow-vs-pytorch-2024.png" width="150"/>


In [3]:
import torch

print("Versión de PyTorch:", torch.__version__)
print("GPU disponible:", "Sí" if torch.cuda.is_available() else "No")

Versión de PyTorch: 2.9.0+cpu
GPU disponible: No


## **jax.numpy**

![](https://miro.medium.com/v2/resize:fit:720/format:webp/1*G5677kOYPHyEzEGZaJ33sA.png)

[jax.numpy](https://docs.jax.dev/en/latest/jax.numpy.html) es como NumPy, pero acelerado y diferenciable, y jnp es solo el nombre corto que usamos para llamar a esas funciones.

A diferencia de Numpy, que solo se ejecuta en CPU, JAX admite operaciones de matriz en múltiples aceleradores, incluyendo CPU, GPU y TPU. Esta capacidad permite a JAX gestionar eficientemente cálculos a gran escala y tareas de aprendizaje profundo, aprovechando el procesamiento paralelo para aumentar significativamente el rendimiento.

> Esta sección está basada en: [jax-vs-numpy-key-differences-and-benefits](https://medium.com/@harshavardhangv/jax-vs-numpy-key-differences-and-benefits-72e442bbf67f)


### **Consideraciones importantes al usar JAX**

#### 1. Inmutabilidad
   - Los arrays de JAX son **inmutables**. No se pueden modificar directamente como en NumPy.
   - Para actualizar valores específicos se usan `.at[index].set(valor)`:

In [7]:
array = jnp.array([ 1 , 2 , 3 , 4 ])

# La asignación directa como en NumPy generará un error
# array[1] = 3 # Descomentar esta línea generará un error

# Usa .at para especificar el índice y .set para actualizar el valor
array = array.at[ 1 ]. set ( 3 )

# Imprime la matriz actualizada
print(array)   # Salida: [1, 3, 3, 4]

[1 3 3 4]


#### 2. Funciones puras

   * JAX requiere que las funciones sean **puras**: siempre devuelven la misma salida para la misma entrada y no modifican estados externos.
   * JAX no puede compilar funciones que alteren variables globales
  

In [8]:
# Define una variable global
global_var = 0

# Define una función no pura que modifica la variable global
def  impure_function ( x ):
    global global_var
    global_var = x
    return x

# Compilación JIT de JAX
jitted_function = jax.jit(impure_function)
print (jitted_function( 5.0 ))

# Imprime la variable global
print ( "Variable global:" , global_var)

5.0
Variable global: JitTracer(~float32[])


El resultado anterior se debe a que jax.jitla compilación espera una función pura pero encontró una función no pura y esto arrojó un resultado inesperado.

#### 3. Indexación fuera de límites

   * NumPy lanza `IndexError` si un índice está fuera de rango. JAX ajusta el índice al límite válido:

In [10]:
# Numpy
# Inicializar la matriz
array = np.array([ 1 , 2 , 3 , 4 ])

# imprimir el índice fuera de los límites
# print(array[ 6 ]) # genera un IndexError

# JAX
# Inicializar la matriz
array = jnp.array([ 1 , 2 , 3 , 4 ])

# imprimir el índice fuera de los límites
print(array[ 6 ], array[- 3 ]) # imprime (4, 1)

4 2


#### 4. Entradas deben ser arrays

* JAX no acepta directamente listas o tuplas en operaciones numéricas.
* Hay que **transformarlas a un array de JAX (`jnp.array`)** antes de usarlas.


In [11]:
lista = [1, 2, 3]
array = jnp.array(lista)
print(jnp.sum(array))  # 6

6


#### 5. Tipos de datos

Los tipos de datos son prácticamente iguales a NumPy.

### **Ejemplo práctico**

En este bloque realizaremos **pruebas prácticas con `jax.numpy`**, la implementación de NumPy dentro de JAX.



#### Crear arrays

In [13]:
array1 = jnp.array([1, 2, 3, 4])
array2 = jnp.array([[1, 2], [3, 4]])

print("Array 1:", array1)
print("Array 2:\n", array2)

Array 1: [1 2 3 4]
Array 2:
 [[1 2]
 [3 4]]


#### Operaciones

In [17]:
print("Array:", array1)
print("Suma de array1:", jnp.sum(array1))
print("Media de array1:", jnp.mean(array1))
print("Mediana de array1:", jnp.median(array1))

# Moda (no existe directa, se calcula así)
values, counts = jnp.unique(array1, return_counts=True)
mode = values[jnp.argmax(counts)]
print("Moda de array1:", mode)

print("Producto de array1:", jnp.prod(array1))
print("Máximo de array1:", jnp.max(array1))
print("Mínimo de array1:", jnp.min(array1))
print("Desviación estándar de array1:", jnp.std(array1))
print("Varianza de array1:", jnp.var(array1))
print("Suma acumulada de array1:", jnp.cumsum(array1))
print("Producto acumulado de array1:", jnp.cumprod(array1))
print("Producto punto de array1 consigo mismo:", jnp.dot(array1, array1))
print("Array ordenado:", jnp.sort(array1))
print("Raíz cuadrada de array1:", jnp.sqrt(array1))
print("Array al cuadrado:", jnp.power(array1, 2))
print("Valor absoluto de array1:", jnp.abs(array1))
print("Array redondeado:", jnp.round(array1))

Array: [1 2 3 4]
Suma de array1: 10
Media de array1: 2.5
Mediana de array1: 2.5
Moda de array1: 1
Producto de array1: 24
Máximo de array1: 4
Mínimo de array1: 1
Desviación estándar de array1: 1.118034
Varianza de array1: 1.25
Suma acumulada de array1: [ 1  3  6 10]
Producto acumulado de array1: [ 1  2  6 24]
Producto punto de array1 consigo mismo: 30
Array ordenado: [1 2 3 4]
Raíz cuadrada de array1: [1.        1.4142135 1.7320508 2.       ]
Array al cuadrado: [ 1  4  9 16]
Valor absoluto de array1: [1 2 3 4]
Array redondeado: [1 2 3 4]


#### Indexación y slicing

In [19]:
print("Primer elemento de array1:", array1[0])
print("Últimos dos elementos de array1:", array1[-2:])
print("Fila 0 de array2:", array2[0])

Primer elemento de array1: 1
Últimos dos elementos de array1: [3 4]
Fila 0 de array2: [1 2]


#### Inmutabilidad - Actualizar valores usando `.at[].set()`

In [20]:
# array1[1] = 10  # Esto daría error

In [21]:
array1_updated = array1.at[1].set(10)
print("\nArray1 actualizado:", array1_updated)


Array1 actualizado: [ 1 10  3  4]


## **Bibliografia**

- [jax](https://docs.jax.dev/en/latest/)
- [chatgpt](https://chatgpt.com/)
- [eiposgrados](https://eiposgrados.com/blog-python/jax-machine-learning/#:~:text=JAX%20es%20una%20nueva%20biblioteca,flexible%20y%20f%C3%A1cil%20de%20aprender.)
- [medium](https://medium.com/@harshavardhangv/jax-vs-numpy-key-differences-and-benefits-72e442bbf67f)